
# 1. OpenAI VLM (GPT) - Basics
This section demonstrates the basic usage of OpenAI's Vision Language Model (VLM) capabilities using GPT-4.1.
We will use the OpenAI API to analyze an image and provide detailed textual insights.

**Support Material**

- https://platform.openai.com/docs/quickstart
- https://platform.openai.com/docs/guides/text
- https://platform.openai.com/docs/guides/images-vision?api-mode=chat
- https://platform.openai.com/docs/guides/structured-outputs


In [9]:
import openai
from dotenv import load_dotenv  
import base64
import json
import textwrap

# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

load_dotenv()
openAIclient = openai.OpenAI()

# Path to your image
img = "images/street_scene.jpg"

In [ ]:
#basic call to gpt with prompt and image
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)


# Wrap the text to a specified width
response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

The image depicts a lively urban street scene with a mix of pedestrians and vehicles. In the foreground, there is a
wooden bench on a city sidewalk where an elderly man in a suit is sitting, and next to him, a young woman is reading a
newspaper. Nearby, a young boy is seated on the ground, focused on a tablet or smartphone, and another person is lying
on the pavement, dressed in casual clothing including a red jacket and jeans.  Several pigeons are scattered around this
area. In the street crossing, a man riding a bicycle, a person on a scooter, and a musician playing a guitar are
visible. Cars and a taxi are driving through the intersection. Buildings with glass windows and storefronts line the
street, and the scene appears to be set during the day, with sunlight illuminating the area and skyscrapers in the
background. There are also a few people walking on the sidewalk at the right side of the image. The overall atmosphere
feels dynamic and vibrant.



# 1.1 Structured Output
Here, we expand upon the VLM example to request structured outputs. This approach allows for extracting 
well-organized information from images in a machine-readable format, such as JSON.

**Support Material**:
- https://platform.openai.com/docs/guides/structured-outputs


In [11]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={ "type": "json_object" },# NEW!!
    temperature = 0
)

returnValue = completion.choices[0].message.content

We parse the json in a dict structure:

In [12]:
#json. loads() converts JSON strings to Python objects
output = json.loads(returnValue)

So we can access specific infos:

In [13]:
output
#output["foreground"]["people"]

{'scene': 'Urban city street intersection during daytime',
 'background': {'buildings': [{'style': 'Brick facade with green awnings',
    'windows': 'Large, reflecting sunlight',
    'shops': 'Visible with warm lighting inside'},
   {'style': 'Modern glass skyscrapers',
    'height': 'Very tall, extending into the sky',
    'details': 'Reflective surfaces, some with trees in front'},
   {'church': {'architecture': 'Traditional with a steeple',
     'location': 'Center background, between skyscrapers'}}],
  'traffic_light': {'color': 'Yellow',
   'position': 'Hanging over the intersection'}},
 'street': {'crosswalk': 'Wide zebra stripes, busy with activity',
  'vehicles': [{'type': 'Taxi',
    'color': 'White',
    'motion': 'Blurred, indicating movement'},
   {'type': 'SUV', 'color': 'Gray', 'motion': 'Moving slowly'},
   {'type': 'Sedan', 'color': 'Orange', 'motion': 'Blurred, moving fast'}],
  'motorcycles': [{'rider': 'Wearing black gear and white helmet',
    'motion': 'Moving acro


# JSON Schema for Controlled Structured Outputs
In this section, we define a JSON schema for a more controlled and specific output from the model. 
Using this schema, we can ensure the model adheres to predefined data types and structures while describing images.In this case we will provide the json schema directly.



In [14]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={
                "type": "json_schema",    
                "json_schema": {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]
                    }}},
    temperature = 0
)

returnValue = completion.choices[0].message.content

In [15]:
output_image_extraction = json.loads(returnValue)

In [16]:
output_image_extraction["people"]

[{'position': 'sitting on the ground near a flower pot',
  'age': 16,
  'activity': 'using a smartphone',
  'gender': 'male'},
 {'position': 'lying on the ground',
  'age': 18,
  'activity': 'resting or sleeping',
  'gender': 'male'},
 {'position': 'sitting on a bench',
  'age': 65,
  'activity': 'reading a newspaper',
  'gender': 'female'},
 {'position': 'sitting on a bench',
  'age': 70,
  'activity': 'thinking or resting with hand on face',
  'gender': 'male'},
 {'position': 'walking on the sidewalk near the bench',
  'age': 20,
  'activity': 'using a smartphone',
  'gender': 'female'},
 {'position': 'riding a motorcycle in the crosswalk',
  'age': 30,
  'activity': 'riding motorcycle',
  'gender': 'male'},
 {'position': 'walking in the crosswalk playing guitar',
  'age': 25,
  'activity': 'playing guitar',
  'gender': 'male'},
 {'position': 'riding a scooter in the crosswalk',
  'age': 28,
  'activity': 'riding scooter',
  'gender': 'female'},
 {'position': 'walking on the sidewalk

Alternatively: 


OpenAI SDKs for Python and JavaScript also make it easy to define object schemas using Pydantic and Zod respectively. Below, you can see how to extract information from unstructured text that conforms to a schema defined in code.


```python
from pydantic import BaseModel


class Person(BaseModel):
    position: str 
    age: int 
    activity: str 
    gender: str


class ImageExtraction(BaseModel):
    number_of_people: int 
    atmosphere: str 
    hour_of_the_day: int 
    people: list[Person] 

completion = openAIclient.beta.chat.completions.parse(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": "describe the image in detail"}
    ],
    response_format=ImageExtraction,
)

output_image_extraction = completion.choices[0].message.parsed


We can then integrate the extracted information in full or partially in a new prompt for a new extraction

In [17]:
#alert service prompt 

alert_sys_prompt = " you are an experienced first aid paramedical"
alert_prompt= """Extract from the following scene analysis give to you in json format, 
if anyone might be in danger and if the Child Hospital or normal Hospital should be alerted. 
Give the a concise answer
The situation is given to you from this object: """ + str(output_image_extraction)

In [18]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": alert_sys_prompt},
        {"role": "user", "content": alert_prompt}
    ],
)

# Wrap the text to a specified width
response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

No one appears to be in immediate danger based on the scene. No hospital alert is necessary at this time.


In [19]:
completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Considering this list of people"+str(output_image_extraction["people"])+".Identify the youngest in the picture I provide and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(img)}",
                        #"detail": "low"
                    }
                },
            ],
        }
    ],
)

# Wrap the text to a specified width
response = str(completion.choices[0].message.content)
print(textwrap.fill(response, width=120))

The youngest person in the picture, according to the provided list, is the 16-year-old male sitting on the ground near a
flower pot using a smartphone.  Based on the image, his approximate box_2d coordinates normalized to 0-1000 are:  [ymin,
xmin, ymax, xmax] = [755, 144, 930, 310]



# 2. Google VLM (Gemini)
This section demonstrates the use of Google's Vision Language Model, Gemini. 
We explore basic text generation as well as its ability to analyze images and provide relevant outputs.

**Support Material**:
- https://ai.google.dev/gemini-api/docs/quickstart
- https://ai.google.dev/gemini-api/docs/text-generation
- https://ai.google.dev/gemini-api/docs/image-understanding
- https://ai.google.dev/gemini-api/docs/structured-output?example=recipe

In [20]:
%matplotlib inline
from dotenv import load_dotenv  
from google import genai
from PIL import Image
import textwrap
import json

load_dotenv()
client = genai.Client()

# Path to your image
img = "images/street_scene.jpg"

Basic call:

In [21]:
response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works to a 90 years old. in few words"
)

print(textwrap.fill(response.text, width=120))

Imagine it like teaching a very fast student.  You show it *lots* of examples – like showing a child thousands of
pictures of cats until it knows what a cat is.  Then, it can recognize new cats, understand what you say, or help answer
questions, all very quickly and cleverly.


and with images: 

In [22]:
im = Image.open(img)

response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Describe the scene in details\n"],
                                          )

print(textwrap.fill(response.text, width=120))

This image presents a bustling urban street scene, bathed in the warm, golden light of what appears to be late afternoon
or early morning. The perspective is eye-level, looking down a long city street, creating a strong sense of depth.
**Foreground:** The immediate foreground features a concrete sidewalk and a prominent zebra crosswalk. *   To the far
left, a young person with short brown hair sits cross-legged on the pavement, engrossed in a tablet or phone. They wear
a green jacket and grey shorts. Beside them, a rustic wooden pot overflows with vibrant red geraniums. *   Closer to the
center, another young individual, dressed in a red hoodie and blue jeans, lies sprawled casually on their back on the
sidewalk, eyes closed. *   Numerous pigeons are scattered across the sidewalk and crosswalk, some pecking at the ground,
others standing still. *   To the right, a classic wooden park bench provides a moment of respite for two individuals.
An older man in a dark suit sits on the left of

Also here we can extract structured output (Gemini actually prefers pydantic syntax - let's see what happens with a schema as before)-> check limitations in https://ai.google.dev/gemini-api/docs/structured-output?example=recipe 

In [23]:
json_schema = {
                    "name": "img_extract",
                    "schema": {
                    "type": "object",
                    "properties": {
                        "numberOfPeople": {
                        "type":"integer",
                        "description": "The total number of people in the environment",
                        "minimum": 0
                        },
                        "atmosphere": {
                        "type": "string",
                        "description": "Description of the atmosphere, e.g., calm, lively, etc."
                        },
                        "hourOfTheDay": {
                        "type": "integer",
                        "description": "The hour of the day in 24-hour format",
                        "minimum": 0,
                        "maximum": 23
                        },
                        "people": {
                        "type": "array",
                        "description": "List of people and their details",
                        "items": {
                            "type": "object",
                            "properties": {
                            "position": {
                                "type": "string",
                                "description": "Position of the person in the environment, e.g., standing, sitting, etc."
                            },
                            "age": {
                                "type": "integer",
                                "description": "Age of the person",
                                "minimum": 0
                            },
                            "activity": {
                                "type": "string",
                                "description": "Activity the person is engaged in, e.g., reading, talking, etc."
                            },
                            "gender": {
                                "type": "string",
                                "description": "Gender of the person",
                                "enum": ["male", "female", "non-binary", "other", "prefer not to say"]
                            }
                            },
                            "required": ["position", "age", "activity", "gender"]
                        }
                        }
                    },
                    "required": ["numberOfPeople", "atmosphere", "hourOfTheDay", "people"]}}



config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema,
    }


response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Describe the scene in details, follwoing exactly the given json schema\n"],
                                          config=config
                                          )

print(response.text)

[
  {
    "label": "person",
    "box_2d": [197, 600, 359, 891],
    "rot_y": -0.2,
    "occluded": false,
    "truncated": false,
    "alpha": -0.2,
    "score": 1.0
  },
  {
    "label": "person",
    "box_2d": [320, 700, 627, 966],
    "rot_y": 0.0,
    "occluded": false,
    "truncated": false,
    "alpha": 0.0,
    "score": 1.0
  },
  {
    "label": "person",
    "box_2d": [321, 291, 500, 477],
    "rot_y": 0.1,
    "occluded": false,
    "truncated": false,
    "alpha": 0.1,
    "score": 1.0
  },
  {
    "label": "motorcycle",
    "box_2d": [335, 389, 550, 573],
    "rot_y": 0.1,
    "occluded": false,
    "truncated": false,
    "alpha": 0.1,
    "score": 1.0
  },
  {
    "label": "person",
    "box_2d": [498, 315, 612, 532],
    "rot_y": 0.0,
    "occluded": false,
    "truncated": false,
    "alpha": 0.0,
    "score": 1.0
  },
  {
    "label": "guitar",
    "box_2d": [539, 395, 630, 467],
    "rot_y": 0.0,
    "occluded": false,
    "truncated": false,
    "alpha": 0.0,
    "s

Does it match your schema?

Let's try to use Gemini to detect an object in the image and get its coordinates:


In [24]:
prompt = "Identify the youngest in the picture and give me back their coordinates. The box_2d should be [ymin, xmin, ymax, xmax] normalized to 0-1000."

config={"response_mime_type": "application/json"}

response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[img, prompt],
                                          config=config
                                          )

bounding_boxes = json.loads(response.text)
print(bounding_boxes)

{'box_2d': [665, 412, 937, 517]}


Gemini2+ was trained specifically for object detection/ segmentation tasks. More details: https://colab.research.google.com/github/google-gemini/cookbook/blob/main/quickstarts/Spatial_understanding.ipynb


## 3.Extract Structured Infos from Hand-written note - GPT & Gemini

Let’s try **not** to extract structured information from a handwritten note (e.g., `prescription1.jpg`) using **both models**.

Consider the file: `/images/prescription1.jpg`.  
Have a look at it.

### JSON Schema
Let’s define a JSON schema for the extraction task:


In [25]:
json_schema_prescription = {
 "name": "prescription_extract",
"schema": {
  "type": "object",
  "properties": {
    "doctor_name": { "type": "string" },
    "patient_name": { "type": "string" },
    "patient_dob": { "type": "string" },
    "meds": {
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "name": { "type": "string" },
          "dose": { "type": "string" },
          "frequency": { "type": "string" },
          "instructions": { "type": "string" }
        },
        "required": ["name"]
      }
    },
    "signature": { "type": "boolean" }
  },
  "required": ["doctor_name", "patient_name", "meds"]
}}

Extract structured infos using Gemini: 

In [26]:
im = Image.open("images/prescription1.jpg")

config={
        "response_mime_type": "application/json",
        "response_json_schema": json_schema_prescription,
    }


response = client.models.generate_content(model="gemini-2.5-flash",
                                          contents=[im, "Extract infos from image, following the given json schema.\n"],
                                          config=config
                                          )

print(response.text)

{
  "doctor": "Dr. Markus Müller",
  "patient": "Claudie Fischer",
  "date_of_birth": "1978-04-01",
  "gender": "Female",
  "medication": "Ibuprofen",
  "dosage": "400mg",
  "frequency": "3 times daily",
  "instructions": "after eating"
}


If the output is **not valid JSON** and contains extra strings, it must be **parsed** before it can be loaded into a Python dict.  
Below is an example helper function that does this.

> **Note:** Since Gemini returns a Pydantic model, you *could* use Pydantic methods to handle parsing.  
> We avoid that here to keep the workflow generally compatible across models.


In [27]:
import re
import json 
def parse_json_in_output(output):
    """
    Extracts and converts JSON-like data from the given text output to a Python dictionary.
    
    Args:
        output (str): The text output containing the JSON data.
    
    Returns:
        dict: The parsed JSON data as a Python dictionary.
    """
    # Regex to extract JSON-like portion
    json_match = re.search(r"\{.*?\}", output, re.DOTALL)
    if json_match:
        json_str = json_match.group(0)
        # Fix single quotes and ensure proper JSON formatting
        json_str = json_str.replace("'", '"')  # Replace single quotes with double quotes
        try:
            # Convert the fixed JSON string into a dictionary
            json_data = json.loads(json_str)
            return json_data
        except json.JSONDecodeError:
            return "The extracted JSON is still not valid after formatting."
    else:
        return "No JSON data found in the given output."

In [28]:
print(parse_json_in_output(response.text))

{'doctor': 'Dr. Markus Müller', 'patient': 'Claudie Fischer', 'date_of_birth': '1978-04-01', 'gender': 'Female', 'medication': 'Ibuprofen', 'dosage': '400mg', 'frequency': '3 times daily', 'instructions': 'after eating'}


In [29]:
json.loads(response.text)

{'doctor': 'Dr. Markus Müller',
 'patient': 'Claudie Fischer',
 'date_of_birth': '1978-04-01',
 'gender': 'Female',
 'medication': 'Ibuprofen',
 'dosage': '400mg',
 'frequency': '3 times daily',
 'instructions': 'after eating'}

Now let's do the same with GPT

In [30]:
im = "images/prescription1.jpg"

completion = openAIclient.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "you are a careful observer. the response should be in json format"},
        {"role": "user", "content": [
                {"type": "text", "text": "Describe the image in detail"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image(im)}",
                        #"detail": "low"
                    }
                },
            ]}
    ],
    response_format={
                "type": "json_schema",   "json_schema": json_schema_prescription},
    temperature = 0
)

returnValue = completion.choices[0].message.content

In [31]:
returnValue

'{"doctor_name":"Dr. Markus Müller","patient_name":"Claudia Fischer","patient_dob":"1.4.1978","meds":[{"name":"Ibuprofen","dose":"400 mg","frequency":"3x","instructions":"nach dem Essen"}],"signature":true}'

Any difference wiht the output of Gemini vs your schema? 

No need for parsing now. We load the json in a python dict structure with json.loads

In [32]:
print(json.loads(returnValue))

{'doctor_name': 'Dr. Markus Müller', 'patient_name': 'Claudia Fischer', 'patient_dob': '1.4.1978', 'meds': [{'name': 'Ibuprofen', 'dose': '400 mg', 'frequency': '3x', 'instructions': 'nach dem Essen'}], 'signature': True}
